In [ ]:
!hostname

In [ ]:
## Check what is running
# !ps -eo pid,etime,user | grep ^[[:space:]]*[0-9] | grep xyz | awk '{print $1, $2, $3}' | while read pid etime user; do echo -e "$pid\t$etime\t$user\t$(readlink /proc/$pid/cwd)"; done


In [ ]:
from pddl.logic import Predicate, constants, variables
from pddl.core import Domain, Problem, Action, Requirements
from pddl.formatter import domain_to_string, problem_to_string
from pddl import parse_domain, parse_problem

from pddl.logic.effects import When, AndEffect
from pddl.logic.base import Not, And
from pddl.logic.predicates import EqualTo
from pddl.custom_types import name




import copy

import sys, os, csv

from options import Program_Options
import benchmark, planners, translation, hypothesis, simulation
import subprocess

import pandas as pd

import re

import time

import shutil



In [ ]:
def is_human_action_valid_in_robot_model(action_string, filename):
    with open(filename, 'r') as file:
        for line in file:
            if line.strip() == action_string:
                return True
    return False

In [ ]:
def load_obs(domain_folder_path='', file_name='obs.dat') :
    obs = []
    
    if domain_folder_path == '':
        obs_path = f"{file_name}"
    else:
        obs_path = f"{domain_folder_path}/{file_name}"
        
    
    
    instream = open( obs_path )
    for line in instream :
        line = line.strip()
        obs.append( line )
    instream.close()
    return obs

In [ ]:
def load_hypotheses(domain_folder_path, a_planner_type, domain_name=None) :

    hyps = []

    hyps_path = f"{domain_folder_path}/hyps.dat"
    
    instream = open( hyps_path )

    for line in instream :
        line = line.strip()
        H = hypothesis.Probabilistic(domain_folder_path, a_planner_type, domain_name)
        H.atoms = [  tok.strip() for tok in line.split(',') ]
        hyps.append( H )

    instream.close()

    return hyps

In [ ]:
def get_prob_template_for_hyps (domain_folder_path, problem_file):    
    problem_file = parse_problem ( problem_file)
    problem_file_string = problem_to_string(problem_file)
    # Find the position of '(:goal'
    start_pos = problem_file_string.find('(:goal')

    # Check if '(:goal' was found
    if start_pos != -1:
        # Replace everything after '(:goal' with '<HYPOTHESIS>)))'
        problem_file_string = problem_file_string[:start_pos] + '(:goal \n(and \n \t<HYPOTHESIS> \n)))'

    with open(f"{domain_folder_path}/template.pddl", 'w') as file:
        file.write(problem_file_string)

In [ ]:
def move_files_with_suffix(source, destination, suffix):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination):
        os.makedirs(destination)

    for item in source:
        # Get the name of the item (file or folder) without the path
        item_name = os.path.basename(item)

        # Split the file name into name and extension parts
        item_name_root, item_extension = os.path.splitext(item_name)

        # If the item has an extension, add the suffix before the extension
        # If not, just append the suffix at the end
        new_name = item_name_root + suffix + item_extension

        # Generate the new path in the destination folder
        new_path = os.path.join(destination, new_name)

        # Move the item to the destination folder
        # If the destination already exists but is not a directory,
        # this may replace it depending on the platform
        shutil.move(item, new_path)

In [ ]:
domain_folder_path_list=['xyz']

In [ ]:
for domain_folder_path in domain_folder_path_list[4:5]:
    print(domain_folder_path)

### For Observed Actions

In [ ]:
planner_type_list = ['optimal', 'non_optimal']


domain = '6_Gripper'

#dont pass domain name for the domains without mixed types
# domain_name = 'logistics'

for a_planner_type in planner_type_list:
    for domain_folder_path in domain_folder_path_list:

        instance_name = domain_folder_path.split('/')[-1]
        print(instance_name)
        print(domain_folder_path)

        #dont pass domain name for the domains without mixed types
        hyps = load_hypotheses(domain_folder_path, a_planner_type)


        observer = simulation.Observer( hyps )


        real_obs = load_obs(domain_folder_path=domain_folder_path)
        probs = []
        costs = []
        likelihoods = []


        probs_possible_obs = []
        costs_possible_obs = []
        likelihoods_possible_obs = []

        options = []

        human_domain = f"{domain_folder_path}/human_domain.pddl"

        column_names = ['planner','domain',
            'instance_name',
            'Real_Time_Slice',
            'Real_Obs',
            'Possible_Next_Obs',
            'P(O|Gf)',
            'P(~O|Gf)',
            'P(O|~Gf)',
            'P(~O|~Gf)',
            'P(Gf|O)',
            'P(~Gf|O)',
            'Cost_Gf_with_O',
            'Cost_Gf_with_~O',
            'Cost_~Gf_with_O',
            'Cost_~Gf_with_~O',
            'P(Oi|O,Gf)',
            'P(~Oi|O,Gf)',
            'P(Oi|O,~Gf)',
            'P(~Oi|O,~Gf)',
            'P(Gf|O+Oi)',
            'P(~Gf|O+Oi)',
            'next_Cost_Gf_with_O',
            'next_Cost_Gf_with_~O',
            'next_Cost_~Gf_with_O',
            'next_Cost_~Gf_with_~O',
            'binary_unit_cost_2',
            'real_obs_total_time',
            'next_obs_total_time',
        ]

        summary_column_names = [

            'Ratio',
            'Real_Time_Slice',
            'Real_Obs',
            'cost_of_exp_both_method',
            'cost_method_1',
            'cost_method_2'
        ]

        df_row_lst = []

        df_row_lst_2 = []

        # create an empty DataFrame
        df = pd.DataFrame(columns=column_names)


        ratio_list = [1.0/5.0, 1.0/4.0, 1.0/3.0, 1.0/2.0, 1.0, 2.0, 3.0, 4.0, 5.0 ]

        for j in range(0, len(real_obs) ) :

            robot_domain = f"{domain_folder_path}/robot_domain.pddl"
            robot_init_problem = f"{domain_folder_path}/robot_problem.pddl" 

            #crete problem template for hyps
            get_prob_template_for_hyps(domain_folder_path, f"{domain_folder_path}/merged_problem.pddl")

            #observer.obs is a list of obs until time j+1
            observer.obs = real_obs[0:j+1]



            with open('temp_obs.dat', 'w') as file:
                file.write('\n'.join(observer.obs))  # Write all elements except the last one



            #1) Early stop: Check if human observed action is valid in the robot model
            #first we need to list valid robot actions before executing obs.
            #if we can execute it then we need to get updated robot problem file




            #Step 1: Get possible human actions at this obs step
            if j==0:
                #dont pass domain name for the domains without mixed types    
                os.system(f"python t_get_next_possible_actions_list.py -d {robot_domain} -i {robot_init_problem}")
            else:
                os.system(f"python t_get_next_possible_actions_list.py -d {robot_domain} -i temp_iterated_problem_robot_at_{j}.pddl")

            robot_valid_poss_act_at_this_step_list = f'{instance_name}_robot_possible_actions_at_{j}.dat'

            os.rename('temp_next_possible_actions.dat',robot_valid_poss_act_at_this_step_list)

            human_action_at_this_step  = real_obs[j]

            print('Step and human action', j+1,human_action_at_this_step)


            #Step 2: Check if the observed human action is valid in robot model
            is_valid_act_in_human_model = is_human_action_valid_in_robot_model(human_action_at_this_step, robot_valid_poss_act_at_this_step_list)
            print('is_valid_act_in_human_model:', is_valid_act_in_human_model)

            flag_is_obs_valid_in_robot ='not yet'



            if is_valid_act_in_human_model:
                temp_obs = 'temp_obs.dat'

                #This creates a problem file called temp_iterated_problem.pddl
                #this is how we can iterate the initial state based on the obs sequence
                os.system(f"python t_upd_prob_file_with_obs_list_not_merged.py -d {robot_domain} -i {robot_init_problem} -o {temp_obs}")        
                #this is j+1 bc we'll use this for the next step, current problem has been used already
                os.rename('temp_iterated_problem.pddl',f'temp_iterated_problem_robot_at_{j+1}.pddl')




                ###new binary calculation
                os.system(f"python t_get_next_possible_actions_list.py -d {robot_domain} -i temp_iterated_problem_robot_at_{j+1}.pddl")
                robot_valid_poss_act_at_next_step_list = f'{instance_name}_robot_next_possible_actions_at_{j}.dat'

                os.rename('temp_next_possible_actions.dat',robot_valid_poss_act_at_next_step_list)


            else:
                flag_is_obs_valid_in_robot = 'No'

            #------




            if flag_is_obs_valid_in_robot != 'No':

                # Start the timer
                start_time = time.time()

                observer.test( options )

                # End the timer
                end_time = time.time()

                real_obs_total_time =  end_time - start_time

                source_folders_files = ['prob-0-PR', 'prob-1-PR', 'pr-problem-0-neg-O.log','pr-problem-0-O.log','pr-problem-1-neg-O.log','pr-problem-1-O.log']
                destination_folder = f'.saved_files/{instance_name}/{a_planner_type}/0-real_obs'
                suffix_to_add = f'_time_{j}'
                move_files_with_suffix(source_folders_files, destination_folder, suffix_to_add)

                os.system( 'rm -rf prob-*-PR' )
                #P(O|G)
                posterior_probs = [ h.Probability_O for h in hyps ]

                #1. P(O|Gf)
                prob_O_given_goal_fail= hyps[0].Probability_O

                #2. P(-O|Gf)
                prob_Not_O_given_goal_fail= hyps[0].Probability_Not_O

                #3. P(O|-Gf)
                prob_O_given_Not_goal_fail= hyps[1].Probability_O

                #4. P(-O|-Gf)
                prob_Not_O_given_Not_goal_fail= hyps[1].Probability_Not_O

                likelihoods.append( [ '%s/%s'%(h.Probability_O,h.Probability_Not_O) for h in hyps ] )
                sum = 0.0
                for p in posterior_probs : sum += p
                # P(G|O)
                total_time = 0
                for h in hyps :
                    total_time += h.total_time
                try :
                    probs.append( [ h.Probability_O/sum for h in hyps ] + [str(total_time)] )

                    #5. P(Gf|O)
                    prob_goal_fail_given_O= (hyps[0].Probability_O)/sum

                    #6. P(¬Gf|O)
                    prob_Not_goal_fail_given_O= (hyps[1].Probability_O)/sum



                except:
                    print("All P(O|G) = 0!!!")

                costs.append( [ '/'.join([ str(h.costs['O']), str(h.costs['neg-O'])]) for h in hyps ] )


                #COSTS for real observation:
                #for goal_failures
                Cost_Gf_with_O = hyps[0].costs['O']
                Cost_Gf_with_neg_O = hyps[0].costs['neg-O']

                #for not goal failures
                Cost_not_Gf_with_O = hyps[1].costs['O']
                Cost_not_Gf_with_neg_O = hyps[1].costs['neg-O']


                #dont run rest if P(Gf|O) > P(¬Gf|O)
                if prob_goal_fail_given_O > prob_Not_goal_fail_given_O:


                    # append new data to dataframe
                    data_to_append = [a_planner_type,domain,instance_name,
                        j+1, #real observation time = real obs number (starts from 1)
                        real_obs[j], #extra info to store the real obs/act
                        'P(Gf|O) > P(¬Gf|O)', 
                        prob_O_given_goal_fail,
                        prob_Not_O_given_goal_fail,
                        prob_O_given_Not_goal_fail,
                        prob_Not_O_given_Not_goal_fail,
                        prob_goal_fail_given_O,
                        prob_Not_goal_fail_given_O,
                        Cost_Gf_with_O,
                        Cost_Gf_with_neg_O,
                        Cost_not_Gf_with_O,
                        Cost_not_Gf_with_neg_O,
                        0.99,
                        0.99,
                        0.99,
                        0.99,
                        0.99,
                        99,
                        99,
                        99,
                        99,
                        99,
                        1,
                        real_obs_total_time,
                        'NA'
                    ]        


                    df_row_lst.append(data_to_append)

                    # create extension
                    df_extended = pd.DataFrame(df_row_lst, columns=column_names)
                    # concatenate to original
                    out_df = pd.concat([df, df_extended])
                    out_df.to_csv(f't_output_{instance_name}_{a_planner_type}.csv', index=False)  # Specify the desired filename               



                    print('At Obs no :', j+1, ' we P(Gf|O) > P(¬Gf|O)')
                    #dont run the next observations
                    break

                temp_domain = f"{domain_folder_path}/merged_domain_with_constants.pddl"
                temp_problem = f"{domain_folder_path}/merged_problem.pddl"
                temp_obs = 'temp_obs.dat'

                #This creates a problem file called temp_iterated_problem.pddl
                #this is how we can iterate the initial state based on the obs sequence
                os.system(f"python t_upd_prob_file_with_obs_list.py -d {temp_domain} -i {temp_problem} -o {temp_obs}")


                #crete a new prob templete for executed realobs sequence
                get_prob_template_for_hyps(domain_folder_path, 'temp_iterated_problem.pddl')



                #Now all the possible next actions will be obs candidate
                #Therefore t_pr2plan needs to be called for each of the obs candidates
                #But first, lets get all the possible obs/action candidates by calling t_get_next_possible_actions_list.py

                #this is how we can get all possible next actions 
                #based on iterated prob file after executing real obs sequence
                #t_get_next_possible_actions_list.py outputs temp_next_possible_actions.dat file 

                #dont pass domain name for the domains without mixed types
                os.system(f"python t_get_next_possible_actions_list.py -d {human_domain} -i temp_iterated_problem.pddl")

                next_possible_obs = load_obs(file_name='temp_next_possible_actions.dat')



                print('This is j: ',j)

                source_folders_files = ['temp_obs.dat','temp_iterated_problem.pddl', 'temp_next_possible_actions.dat']
                destination_folder = f'.saved_files/{instance_name}/{a_planner_type}/0-real_obs'
                suffix_to_add = f'_time_{j}'
                move_files_with_suffix(source_folders_files, destination_folder, suffix_to_add)

                for i in range(0, len(next_possible_obs) ) :


                    ###new binary cost calculation
                    human_possible_next_action = next_possible_obs[i]
                    is_next_obs_valid_in_robot = is_human_action_valid_in_robot_model(human_possible_next_action, robot_valid_poss_act_at_next_step_list)


                    #dont pass domain name for the domains without mixed types
                    hyps_for_possible_obs = load_hypotheses(domain_folder_path, a_planner_type)
                    possible_observer = simulation.Observer( hyps_for_possible_obs )

                    print('This is i: ',i)


                    possible_observer.obs = [next_possible_obs[i]]

                    # Start the timer
                    start_time = time.time()

                    possible_observer.test( options )


    #                 End the timer
                    end_time = time.time()

                    next_obs_total_time =  end_time - start_time

                    source_folders_files = ['prob-0-PR', 'prob-1-PR', 'pr-problem-0-neg-O.log','pr-problem-0-O.log','pr-problem-1-neg-O.log','pr-problem-1-O.log']
                    destination_folder = f'.saved_files/{instance_name}/{a_planner_type}/1-next_obs_candidates/time_{j}'
                    suffix_to_add = f'_candidate_{i}'
                    move_files_with_suffix(source_folders_files, destination_folder, suffix_to_add)

                    os.system( 'rm -rf prob-*-PR' )
                    #P(O|G)
                    posterior_probs = [ h.Probability_O for h in hyps_for_possible_obs ]



                    #2.1. P(Oi|O,Gf)
                    next_prob_O_given_goal_fail= hyps_for_possible_obs[0].Probability_O

                    #2.2. P(-Oi|O,Gf)
                    next_prob_Not_O_given_goal_fail= hyps_for_possible_obs[0].Probability_Not_O

                    #2.3. P(Oi|O,-Gf)
                    next_prob_O_given_Not_goal_fail= hyps_for_possible_obs[1].Probability_O

                    #2.4. P(-Oi|O,-Gf)
                    next_prob_Not_O_given_Not_goal_fail= hyps_for_possible_obs[1].Probability_Not_O


                    likelihoods_possible_obs.append( [ '%s/%s'%(h.Probability_O,h.Probability_Not_O) for h in hyps_for_possible_obs ] )
                    sum = 0.0
                    for p in posterior_probs : sum += p
                    # P(G|O)
                    total_time = 0
                    for h in hyps_for_possible_obs :
                        total_time += h.total_time
                    try :
                        probs_possible_obs.append( [ h.Probability_O/sum for h in hyps_for_possible_obs ] + [str(total_time)] )

                        #2.5. P(Gf|O+Oi)
                        next_prob_goal_fail_given_O= (hyps_for_possible_obs[0].Probability_O)/sum

                        #2.6. P(¬Gf|O+Oi)
                        next_prob_Not_goal_fail_given_O= (hyps_for_possible_obs[1].Probability_O)/sum

                    except ZeroDivisionError :
                        print("All P(O|G) = 0!!!")
                        print("P(O|G) = ", posterior_probs)
                        print("Costs: ", ','.join(['/'.join([ str(h.costs['O']), str(h.costs['neg-O'])]) for h in hyps_for_possible_obs]))
                        probs_possible_obs.append( [ 0.0 for h in hyps_for_possible_obs ] + [str(total_time)] )
            #         print (probs_possible_obs)
                    costs_possible_obs.append( [ '/'.join([ str(h.costs['O']), str(h.costs['neg-O'])]) for h in hyps_for_possible_obs ] )

                    #for goal failures
                    next_Cost_Gf_with_O = hyps_for_possible_obs[0].costs['O']
                    next_Cost_Gf_with_neg_O = hyps_for_possible_obs[0].costs['neg-O']

                    #for not goal failures
                    next_Cost_not_Gf_with_O = hyps_for_possible_obs[1].costs['O']
                    next_Cost_not_Gf_with_neg_O = hyps_for_possible_obs[1].costs['neg-O']

                    binary_unit_cost_2 = 0

                    ###new binary cost calculation
                    #10000000
    #                 if next_Cost_Gf_with_neg_O == 10000000 or next_Cost_not_Gf_with_O == 10000000:
                    if is_next_obs_valid_in_robot == False:
                        binary_unit_cost_2 = 1

                    # append new data to dataframe
                    data_to_append = [a_planner_type, domain,instance_name,
                        j+1, #real observation time = real obs number (starts from 1)
                        real_obs[j], #extra info to store the real obs/act
                        next_possible_obs[i], 
                        prob_O_given_goal_fail,
                        prob_Not_O_given_goal_fail,
                        prob_O_given_Not_goal_fail,
                        prob_Not_O_given_Not_goal_fail,
                        prob_goal_fail_given_O,
                        prob_Not_goal_fail_given_O,
                        Cost_Gf_with_O,
                        Cost_Gf_with_neg_O,
                        Cost_not_Gf_with_O,
                        Cost_not_Gf_with_neg_O,
                        next_prob_O_given_goal_fail,
                        next_prob_Not_O_given_goal_fail,
                        next_prob_O_given_Not_goal_fail,
                        next_prob_Not_O_given_Not_goal_fail,
                        next_prob_goal_fail_given_O,
                        next_prob_Not_goal_fail_given_O,
                        next_Cost_Gf_with_O,
                        next_Cost_Gf_with_neg_O,
                        next_Cost_not_Gf_with_O,
                        next_Cost_not_Gf_with_neg_O,
                        binary_unit_cost_2,
                        real_obs_total_time,
                        next_obs_total_time
                    ]        


                    df_row_lst.append(data_to_append)

                    # create extension
                    df_extended = pd.DataFrame(df_row_lst, columns=column_names)
                    # concatenate to original
                    out_df = pd.concat([df, df_extended])
                    out_df.to_csv(f't_output_{instance_name}_{a_planner_type}.csv', index=False)  # Specify the desired filename


            #if human obs action is not valid in robot model
            else:

                reason= f"Obs no :, {j+1}, is not valid in robot model"
                # append new data to dataframe
                data_to_append = [a_planner_type, domain,instance_name,
                    j+1, #real observation time = real obs number (starts from 1)
                    real_obs[j], #extra info to store the real obs/act
                    reason, 
                    0.99,
                    0.99,
                    0.99,
                    0.99,
                    1.00,
                    0.00,
                    0.99,
                    0.99,
                    0.99,
                    0.99,
                    0.99,
                    0.99,
                    0.99,
                    0.99,
                    0.99,
                    99,
                    99,
                    99,
                    99,
                    99,
                    1,
                    99,
                    'NA'
                ]        


                df_row_lst.append(data_to_append)

                # create extension
                df_extended = pd.DataFrame(df_row_lst, columns=column_names)
                # concatenate to original
                out_df = pd.concat([df, df_extended])
                out_df.to_csv(f't_output_{instance_name}_{a_planner_type}.csv', index=False)  # Specify the desired filename 

                print('Obs no :', j+1, ' is not valid in robot model')

                #don't run the rest of the real obs
                break


        out_df['norm_P(Oi|O,Gf)'] = out_df.groupby('Real_Time_Slice')['P(Oi|O,Gf)'].transform(lambda x: x / x.sum())

        out_df['row_cost_2'] = out_df['norm_P(Oi|O,Gf)'] * out_df['P(Gf|O)'] * out_df['binary_unit_cost_2']



        #'C(G_f)_1'
        #unit cost of failure
        C_f = 1
        out_df['C(G_f)_1'] = out_df['P(Gf|O)']*C_f
        out_df['C(G_f)_2'] = out_df.groupby('Real_Time_Slice')['row_cost_2'].transform('sum') 


        out_df.to_csv(f't_output_{instance_name}_{a_planner_type}.csv', index=False)  # Specify the desired filename



    

In [ ]:
i

In [ ]:
human_action_at_this_step